In [1]:
#invite people for the Kaggle party
import pandas as pd
import numpy as np
import warnings
import datetime
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
train = pd.read_csv('./train.csv.gz',index_col=0)
test = pd.read_csv('./test2.csv.gz',index_col=0)
print(train.shape)
print(test.shape)

FileNotFoundError: [Errno 2] No such file or directory: './train.csv.gz'

In [3]:
### 看看数据当中有没有值都一样的column，这种column没有用，删掉
print('train')
for col in train.columns: # 每一列
    n_unique = train[col].nunique() #该列当中唯一值的个数
    if n_unique == 1:
        train = train.drop([col],axis=1)
        test = test.drop([col],axis=1)
        print(col, 'is unique') #重点
print()
print('test')
for col in test.columns: # 每一列
    n_unique = test[col].nunique() #该列当中唯一值的个数
    if n_unique == 1:
        train = train.drop([col],axis=1)
        test = test.drop([col],axis=1)
        print(col, 'is unique') #重点
print(train.shape)
print(test.shape)

In [4]:
print('train')
print(train.select_dtypes(include='object').isnull().sum()[train.select_dtypes(include='object').isnull().sum()>0])
print()
print('test')
print(test.select_dtypes(include='object').isnull().sum()[test.select_dtypes(include='object').isnull().sum()>0])

In [5]:
for col in ['seller_state', 'seller_province', 'buyer_state', 'buyer_zip_code', 
            'event_home_team', 'event_primary_grouping', 'zone', 
            'section', 'row_descr', 'repeat_buyer_type', 
            'since_last_purchase', 'user_agent', 'browser', 
            'os_type', 'promotion']:
    train[col]=train[col].fillna('None')
    test[col]=test[col].fillna('None')
print(train.isnull().sum().sum())
print(test.isnull().sum().sum())
print(train.columns)

In [6]:
cols = train.columns.tolist()
cols = cols[-1:] + cols[:-1]
train = train[cols]
print(train.columns)

In [7]:
dataset = pd.concat([train,test], sort=False)
print(dataset.shape)

In [8]:
from sklearn.preprocessing import LabelEncoder
for col in ['listing_source', 'buyer_cbt']:
    print(col)
    encodered = LabelEncoder().fit(np.array(dataset[col].astype(str).to_list()))
    dataset['encode_' + col] = encodered.transform(dataset[col].astype(str).values)
    print(dataset['encode_' + col].value_counts())
    print()

In [9]:
# seller join days
seller_join_datetime = pd.to_datetime(dataset['sale_timestamp'],utc=True).sub(pd.to_datetime(dataset['seller_join_date'],utc=True))
dataset['seller_join_days'] = seller_join_datetime/np.timedelta64(1,'D')
print(dataset['seller_join_days'].value_counts())

In [10]:
# inhand days since sale
dataset['sale_to_inhand_days'] = pd.to_datetime(dataset['inhand_date']).sub(pd.to_datetime(dataset['sale_timestamp']), axis=0).dt.days
print(dataset['sale_to_inhand_days'].value_counts())
# days among sale, inhand, event
def func_inhand_to_event_days(row):
    if row['sale_to_inhand_days'] < 0:
        return row['sale_to_event_days']
    else:
        return row['inhand_to_event_days']
dataset['sale_to_event_days'] = pd.to_datetime(dataset['event_timestamp']).sub(pd.to_datetime(dataset['sale_timestamp']), axis=0).dt.days
print(dataset['sale_to_event_days'].value_counts())
print()
dataset['inhand_to_event_days'] = pd.to_datetime(dataset['event_timestamp']).sub(pd.to_datetime(dataset['inhand_date']), axis=0).dt.days
print(dataset['inhand_to_event_days'].value_counts())

In [11]:
# sale date feature
sale_date = pd.DatetimeIndex(pd.to_datetime(dataset['sale_timestamp'],utc=True))
dataset['_sale_hour'] = sale_date.hour
print(dataset['_sale_hour'].value_counts())
dataset['_sale_day'] = sale_date.day
print(dataset['_sale_day'].value_counts())
dataset['_sale_weekday'] = sale_date.weekday
print(dataset['_sale_weekday'].value_counts())
dataset['_sale_month'] = sale_date.month
print(dataset['_sale_month'].value_counts())
dataset['_sale_year'] = sale_date.year
print(dataset['_sale_year'].value_counts())

In [12]:
# event date feature
event_date = pd.DatetimeIndex(pd.to_datetime(dataset['event_timestamp'],utc=True))
dataset['_event_day'] = event_date.day
print(dataset['_event_day'].value_counts())
dataset['_event_weekday'] = event_date.weekday
print(dataset['_event_weekday'].value_counts())
dataset['_event_month'] = event_date.month
print(dataset['_event_month'].value_counts())
dataset['_event_year'] = event_date.year
print(dataset['_event_year'].value_counts())

In [13]:
# inhand date feature
inhand_date = pd.DatetimeIndex(pd.to_datetime(dataset['inhand_date'],utc=True))
def func_inhand_day_since_sale(row):
    if row['sale_to_inhand_days'] < 0:
        return row['_sale_day']
    else:
        return row['_inhand_day']
dataset['_inhand_day'] = inhand_date.day
print(dataset['_inhand_day'].value_counts())
dataset['_inhand_day_since_sale'] = dataset.apply(func_inhand_day_since_sale, axis=1)
print(dataset['_inhand_day_since_sale'].value_counts())
print()
def func_inhand_weekday_since_sale(row):
    if row['sale_to_inhand_days'] < 0:
        return row['_sale_weekday']
    else:
        return row['_inhand_weekday']
dataset['_inhand_weekday'] = inhand_date.weekday
print(dataset['_inhand_weekday'].value_counts())
dataset['_inhand_weekday_since_sale'] = dataset.apply(func_inhand_weekday_since_sale, axis=1)
print(dataset['_inhand_weekday_since_sale'].value_counts())
print()
def func_inhand_month_since_sale(row):
    if row['sale_to_inhand_days'] < 0:
        return row['_sale_month']
    else:
        return row['_inhand_month']
dataset['_inhand_month'] = inhand_date.month
print(dataset['_inhand_month'].value_counts())
dataset['_inhand_month_since_sale'] = dataset.apply(func_inhand_month_since_sale, axis=1)
print(dataset['_inhand_month_since_sale'].value_counts())
print()
def func_inhand_year_since_sale(row):
    if row['sale_to_inhand_days'] < 0:
        return row['_sale_year']
    else:
        return row['_inhand_year']
dataset['_inhand_year'] = inhand_date.year
print(dataset['_inhand_year'].value_counts())
dataset['_inhand_year_since_sale'] = dataset.apply(func_inhand_year_since_sale, axis=1)
print(dataset['_inhand_year_since_sale'].value_counts())

In [14]:
import calendar
def func_get_monday(year, month, index):
    c = calendar.Calendar(firstweekday=calendar.SUNDAY)
    monthcal = c.monthdatescalendar(year,month)
    return [day for week in monthcal for day in week if \
                    day.weekday() == calendar.MONDAY and \
                    day.month == month][index].day
def func_get_thursday(year, month, index):
    c = calendar.Calendar(firstweekday=calendar.SUNDAY)
    monthcal = c.monthdatescalendar(year,month)
    return [day for week in monthcal for day in week if \
                    day.weekday() == calendar.THURSDAY and \
                    day.month == month][index].day

In [15]:
def func_is_sale_holiday(row):
    if row['_sale_month'] == 1:
        if row['_sale_day'] in [1-func_get_monday(int(row['_sale_year']), 1, 0), 
                                 func_get_monday(int(row['_sale_year']), 1, 2)]:
            return 1
        else:
            return 0
    elif row['_sale_month'] == 2:
        if row['_sale_day'] == func_get_monday(int(row['_sale_year']), 2, 2):
            return 1
        else:
            return 0
    elif row['_sale_month'] == 5:
        if row['_sale_day'] == func_get_monday(int(row['_sale_year']), 5, -1):
            return 1
        else:
            return 0
    elif row['_sale_month'] == 7:
        if row['_sale_day'] in [4, 5]:
            return 1
        else:
            return 0
    elif row['_sale_month'] == 9:
        if row['_sale_day'] == func_get_monday(int(row['_sale_year']), 9, 0):
            return 1
        else:
            return 0
    elif row['_sale_month'] == 10:
        if row['_sale_day'] == func_get_monday(int(row['_sale_year']), 10, 1):
            return 1
        else:
            return 0
    elif row['_sale_month'] == 11:
        if row['_sale_day'] in [11, func_get_thursday(int(row['_sale_year']), 11, -1)]:
            return 1
        else:
            return 0
    elif row['_sale_month'] == 12:
        if row['_sale_day'] in [18-31]:
            return 1
        else:
            return 0
    else:
        return 0
dataset['_sale_day_holiday'] = 0
dataset['_sale_day_holiday'] = dataset.apply(func_is_sale_holiday, axis=1)
dataset['_sale_day_holiday'].value_counts()

In [16]:
def func_is_inhand_holiday(row):
    if row['_inhand_month'] == 1:
        if row['_inhand_day'] in [1-func_get_monday(int(row['_inhand_year']), 1, 0), 
                                             func_get_monday(int(row['_inhand_year']), 1, 2)]:
            return 1
        else:
            return 0
    elif row['_inhand_month'] == 2:
        if row['_inhand_day'] == func_get_monday(int(row['_inhand_year']), 2, 2):
            return 1
        else:
            return 0
    elif row['_inhand_month'] == 5:
        if row['_inhand_day'] == func_get_monday(int(row['_inhand_year']), 5, -1):
            return 1
        else:
            return 0
    elif row['_inhand_month'] == 7:
        if row['_inhand_day'] in [4, 5]:
            return 1
        else:
            return 0
    elif row['_inhand_month'] == 9:
        if row['_inhand_day'] == func_get_monday(int(row['_inhand_year']), 9, 0):
            return 1
        else:
            return 0
    elif row['_inhand_month'] == 10:
        if row['_inhand_day'] == func_get_monday(int(row['_inhand_year']), 10, 1):
            return 1
        else:
            return 0
    elif row['_inhand_month'] == 11:
        if row['_inhand_day'] in [11, func_get_thursday(int(row['_inhand_year']), 11, -1)]:
            return 1
        else:
            return 0
    elif row['_inhand_month'] == 12:
        if row['_inhand_day'] in [18-31]:
            return 1
        else:
            return 0
    else:
        return 0
dataset['_inhand_day_holiday'] = 0
dataset['_inhand_day_holiday'] = dataset.apply(func_is_inhand_holiday, axis=1)
dataset['_inhand_day_holiday'].value_counts()

In [17]:
def func_is_event_holiday(row):
    if row['_event_month'] == 1:
        if row['_event_day'] in [1-func_get_monday(int(row['_event_year']), 1, 0), 
                                             func_get_monday(int(row['_event_year']), 1, 2)]:
            return 1
        else:
            return 0
    elif row['_event_month'] == 2:
        if row['_event_day'] == func_get_monday(int(row['_event_year']), 2, 2):
            return 1
        else:
            return 0
    elif row['_event_month'] == 5:
        if row['_event_day'] == func_get_monday(int(row['_event_year']), 5, -1):
            return 1
        else:
            return 0
    elif row['_event_month'] == 7:
        if row['_event_day'] in [4, 5]:
            return 1
        else:
            return 0
    elif row['_event_month'] == 9:
        if row['_event_day'] == func_get_monday(int(row['_event_year']), 9, 0):
            return 1
        else:
            return 0
    elif row['_event_month'] == 10:
        if row['_event_day'] == func_get_monday(int(row['_event_year']), 10, 1):
            return 1
        else:
            return 0
    elif row['_event_month'] == 11:
        if row['_event_day'] in [11, func_get_thursday(int(row['_event_year']), 11, -1)]:
            return 1
        else:
            return 0
    elif row['_event_month'] == 12:
        if row['_event_day'] in [18-31]:
            return 1
        else:
            return 0
    else:
        return 0
dataset['_event_day_holiday'] = 0
dataset['_event_day_holiday'] = dataset.apply(func_is_inhand_holiday, axis=1)
dataset['_event_day_holiday'].value_counts()

In [18]:
# 观察一下可能是分类特征的列
for col in dataset.columns: # 每一列
    n_unique = dataset[col].nunique() #该列当中唯一值的个数
    if 2 < n_unique < 35:
        print(col, n_unique) #重点
        print(dataset[col].value_counts())
        print()

In [19]:
# category some feature for one hot
dataset['_event_ucf'] = ['other' if i in ['Fights', 'Rugby', 'Cricket']
                              else i
                              for i in dataset['event_ucf']]
print(dataset['_event_ucf'].value_counts())
print()
dataset['_experience'] = ['Unknown' if i == 'Native App - Kiosk' 
                              else i
                              for i in dataset['experience']]
print(dataset['_experience'].value_counts())
print()
dataset['_seller_tool'] = ['other' if i in ['VIP Access', 'Fortbend Tickets', 'Ticket Exchange USA', 
                                           'Dkagan', 'ETB System', 'Billets CA', 'BeyondSports']
                           else i
                           for i in dataset['seller_tool']]
print(dataset['_seller_tool'].value_counts())

In [20]:
# 观察一下可能是分类特征的列
for col in dataset.columns: # 每一列
    n_unique = dataset[col].nunique() #该列当中唯一值的个数
    if 2 < n_unique < 35:
        print(col, n_unique) #重点

In [21]:
# one hot
dummies_encode = pd.get_dummies(pd.DataFrame(dataset, 
                                             columns=['seller_type', 'time_to_event', 'event_type', 
                                                      'event_gtl', 'issue_type', 'listing_type', 
                                                      'delivery_method', 'seller_segment_group', 'seller_segment',
                                                      '_event_ucf', '_experience', '_seller_tool']))
dataset = pd.concat([dataset,dummies_encode], axis =1, sort=False)
print(dataset.shape)

In [22]:
len_train=len(train)
train=dataset[:len_train]
test=dataset[len_train:]
train.head(1000).to_csv('./watch.csv')
print(train.shape)
print(test.shape)

In [23]:
for col in train.columns:
    if train[col].dtype == bool:
        print(col)
        train = train.drop([col],axis=1)
for col in train.columns:
    if train[col].dtype == object:
        print(col)
        train = train.drop([col],axis=1)
print(train.shape)
print(train.columns)

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=[150,75])
sns.heatmap(train.corr(), annot=True)

In [25]:
# 观察一下可能是分类特征的列
for col in train.columns: # 每一列
    n_unique = train[col].nunique() #该列当中唯一值的个数
    if 2 < n_unique < 35:
        print(col, n_unique) #重点
        print(train[col].value_counts())
        print()

In [26]:
print(train.columns[100:])
print(train.columns[:100])

In [27]:
features = ['seller_total_orders',

            'ticket_cost', 
            'sell_fee', 
            'logistics_fee', 
            'seller_payout', 
            'gms', 
            
#             'encode_listing_source',
#             'encode_buyer_cbt',

            'seller_join_days',
            
            'sale_to_inhand_days',
            'sale_to_event_days',
            'inhand_to_event_days',
            
            '_sale_hour', 
            '_sale_day', 
            '_sale_weekday', 
            
            '_event_weekday', 
            
            '_inhand_day_since_sale', 
            '_inhand_weekday_since_sale', 
            
            '_sale_day_holiday', 
            '_event_day_holiday', 
            '_inhand_day_holiday', 
            
            'seller_type_Indy', 
            'seller_type_Large Seller', 
            
            'time_to_event_25-48 Hours',
            'time_to_event_49-72 Hours', 
            'time_to_event_73 Hours-1 Week',
            'time_to_event_> 1 Week',
            
            'event_type_Regular',
            'event_type_Other', 
            'event_type_Parking', 
            'event_type_Post-Season', 
            'event_type_Pre-Season', 
            
            'event_gtl_Concerts', 
            'event_gtl_Sports', 
            'event_gtl_Theater', 

            '_event_ucf_Concerts', 
            '_event_ucf_Theater', 
            '_event_ucf_College Football', 
            '_event_ucf_All Other Sports', 
            '_event_ucf_College Basketball',  
            '_event_ucf_Music Festival', 
            '_event_ucf_NBA', 
            '_event_ucf_NFL', 
            '_event_ucf_NHL', 
            '_event_ucf_Motorsports', 
            '_event_ucf_Comedy',
            '_event_ucf_Tennis',
            '_event_ucf_MLB',
            '_event_ucf_Soccer',  
            
            '_experience_Desktop', 
            '_experience_Native App',
            
#             'issue_type_No Issue', 
            
#             'listing_type_Bulk', 
#             'listing_type_Relist', 
#             'listing_type_Single', 
            
            '_seller_tool_ULF', 
            '_seller_tool_Auto Processor',
            '_seller_tool_1Ticket',
            '_seller_tool_iPhone App',
            '_seller_tool_TicketUtils',
            '_seller_tool_Cloakify',
            '_seller_tool_TicketAttendant', 
            '_seller_tool_SkyBox', 
            '_seller_tool_StageFront', 
            '_seller_tool_Android App', 
            '_seller_tool_Auto Pilot', 
            '_seller_tool_ASC', 
            '_seller_tool_TicketTechnology', 
            '_seller_tool_Relist - Web', 
            '_seller_tool_StubHub Pro', 
            '_seller_tool_ASC', 
            '_seller_tool_Relist - iPhone App', 
            '_seller_tool_Sell More Tickets', 
            '_seller_tool_YS Tickets', 
            '_seller_tool_ABL', 
            '_seller_tool_STH', 
            '_seller_tool_Other - APIs', 
            '_seller_tool_Sales Pro', 
            '_seller_tool_ZeroHero', 
            '_seller_tool_Kikkoman', 
            
            'delivery_method_UPS 2 Business Days - Intra-USA', 
            'delivery_method_UPS Next Business Day PM - Intra-USA', 
            'delivery_method_UPS Next Business Day - Intra-USA',
            'delivery_method_UPS Worldwide Saver', 
            'delivery_method_UPS Express Saver - Intra-CA', 
            
            'seller_segment_group_B2C', 
            'seller_segment_group_C2C', 
            
            'seller_segment_Attempted Seller', 
            'seller_segment_Business', 
            'seller_segment_Churned Seller',
            'seller_segment_Occasional Consumer', 
            'seller_segment_Pro Consumer', 
            'seller_segment_Regular Consumer'
           ]
pd.Series(features).shape

In [28]:
X_train = train[[col for col in train.columns if col in features]]
# 按照评价标准变换一下
Y_train = (train['target_days']+1).apply(np.log)

X_test = test[[col for col in test.columns if col in features]]
X_train.shape, Y_train.shape, X_test.shape

In [29]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=200,verbose=1, random_state=23).fit(X_train,Y_train)

In [30]:
sampleSubmission = pd.read_csv('./sampleSubmission2.csv.gz',index_col=0)
sampleSubmission['target_days'] = np.round(np.exp(model.predict(X_test))-1)
print(sampleSubmission['target_days'].value_counts())
sampleSubmission.to_csv('./Carlos-1.csv')

In [31]:
# len_train=len(train)
# train=dataset[:len_train]
# test=dataset[len_train:]
# train.to_csv('./train.csv')
# test.to_csv('./test.csv')
# train.head(1000).to_csv('./watch.csv')
# print(train.shape)
# print(test.shape)